In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import ChatOpenAI

In [2]:
class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

In [3]:
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0).bind_tools([Joke])

In [4]:
model.kwargs["tools"]

[{'type': 'function',
  'function': {'name': 'Joke',
   'description': 'Joke to tell user.',
   'parameters': {'type': 'object',
    'properties': {'setup': {'description': 'question to set up a joke',
      'type': 'string'},
     'punchline': {'description': 'answer to resolve the joke',
      'type': 'string'}},
    'required': ['setup', 'punchline']}}}]

In [5]:
prompt = ChatPromptTemplate.from_messages(
    [("system", "You are helpful assistant"), ("user", "{input}")]
)

In [6]:
prompt

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are helpful assistant')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])

## JsonOutputToolsParser

In [7]:
from langchain.output_parsers.openai_tools import JsonOutputToolsParser

In [8]:
parser = JsonOutputToolsParser()

In [9]:
chain = prompt | model | parser

In [10]:
chain.invoke({"input": "tell me a joke"})

[{'type': 'Joke',
  'args': {'setup': "Why couldn't the bicycle stand up by itself?",
   'punchline': 'It was two tired!'}}]

In [11]:
parser = JsonOutputToolsParser(return_id=True)
chain = prompt | model | parser
chain.invoke({"input": "tell me a joke"})

[{'type': 'Joke',
  'args': {'setup': "Why couldn't the bicycle stand up by itself?",
   'punchline': 'It was two tired!'},
  'id': 'call_fvPO0OV2CJip7QhISkQ32dT2'}]

## JsonOutputKeyToolsParser

In [12]:
from typing import List

from langchain.output_parsers.openai_tools import JsonOutputKeyToolsParser

In [13]:
parser = JsonOutputKeyToolsParser(key_name="Joke")

In [14]:
chain = prompt | model | parser

In [15]:
chain.invoke({"input": "tell me a joke"})

[{'setup': "Why couldn't the bicycle stand up by itself?",
  'punchline': 'It was two tired!'}]

In [16]:
parser = JsonOutputKeyToolsParser(key_name="Joke", return_single=True)
chain = prompt | model | parser
chain.invoke({"input": "tell me a joke"})

{'setup': "Why couldn't the bicycle stand up by itself?",
 'punchline': 'It was two tired!'}

## PydanticToolsParser

In [17]:
from langchain.output_parsers.openai_tools import PydanticToolsParser

In [18]:
class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field


parser = PydanticToolsParser(tools=[Joke])

In [19]:
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0).bind_tools([Joke])
chain = prompt | model | parser

In [20]:
chain.invoke({"input": "tell me a joke"})

[Joke(setup="Why couldn't the bicycle stand up by itself?", punchline='Because it was two tired!')]